In [1]:
with open("input.txt", "r") as infile: 
    contents = infile.read().strip()
contents

'424 players; last marble is worth 71482 points'

## Part 1

In [2]:
class Circle:
    def __init__(self): 
        self.list  = [0]
        self.index = 0
        
    def place(self, value):
        if value % 23 == 0:
            score = value
            if self.index >= 7:
                score += self.list.pop(self.index - 7)
                self.index -= 7
            else:
                score += self.list.pop(self.index - 7)
                self.index = len(self.list) + self.index - 7 + 1
            return score
        if len(self.list) == 1: 
            self.list.append(value)
            self.index += 1
        elif self.index == len(self.list) - 2:
            self.list.append(value)
            self.index += 2
        elif self.index == len(self.list) - 1:
            self.list.insert(1, value)
            self.index = 1
        else: 
            self.list.insert(self.index + 2, value)
            self.index += 2
        return 0

    def __repr__(self):
        return str(self.index) + ":" + str([x if i != self.index else "({})".format(x) for (i, x) in enumerate(self.list)])

In [3]:
class Game: 
    def __init__(self, players, last): 
        self.players = players
        self.last    = last
        self.circle  = Circle()
        self.scores  = dict.fromkeys(range(1, players + 1), 0)
        
    def play(self):
        players = range(1, self.players + 1)
        
        for i in range(1, self.last + 1):
            if i % 100000 == 0: 
                print(str(i).ljust(10), end='\r')
            player = players[(i-1) % self.players]
            score  = self.circle.place(i)
            self.scores[player] += score
        return self
    
    @property
    def winner(self):
        score = max(self.scores.values())
        winners = [player for player in self.scores if self.scores[player] == score]
        
        if len(winners) == 1: 
            return "Player {}".format(winners[0]),score
        return ["Player {}".format(x) for x in winners], score

In [4]:
game = Game(9,25).play()
game.winner

('Player 5', 32)

In [5]:
game = Game(424, 71482).play()
game.winner

('Player 160', 408679)

## Part 2

In [6]:
class Circle:
    class Marble:
        def __init__(self, value, left=None, right=None): 
            self.value = value
            self.left  = left
            self.right = right
            
            #reconnect
            if left is not None: 
                left.right = self
            if right is not None: 
                right.left = self
        
        def __repr__(self):
            return str(self.value)
            
    def __init__(self):
        self.current = self.Marble(0)
        self.current.left  = self.current
        self.current.right = self.current
    
    def insert(self, value):
        if value % 23 == 0: 
            return value + self.left(7).pop()
        self.right()
        self.current = self.Marble(value, left=self.current, right=self.current.right)
        return 0
    
    def pop(self):
        current = self.current
        left, right  = self.current.left, self.current.right
        left.right   = right
        right.left   = left
        self.current = right
        return current.value
    
    def right(self, count=1):
        for _ in range(count):
            self.current = self.current.right
        return self
    
    def left(self, count=1): 
        for _ in range(count):
            self.current = self.current.left
        return self
    
    def __len__(self):
        if self.current is None: 
            return 0 
        count   = 1
        current = self.current.right
        while current != self.current: 
            count += 1
            current = current.right
        return count
    
    def __repr__(self):
        output = list()
        for i in range(len(self)):
            output.append(str(self.current))
            self.right()
        return str(output)

In [7]:
class Game: 
    def __init__(self, players, last): 
        self.players = players
        self.last    = last
        self.circle  = Circle()
        self.scores  = dict.fromkeys(range(1, players + 1), 0)
        
    def play(self):
        players = range(1, self.players + 1)
        
        for i in range(1, self.last + 1):
            player = players[(i-1) % self.players]
            self.scores[player] += self.circle.insert(i)
        return self
    
    @property
    def winner(self):
        score = max(self.scores.values())
        winners = [player for player in self.scores if self.scores[player] == score]
        
        if len(winners) == 1: 
            return "Player {}".format(winners[0]),score
        return ["Player {}".format(x) for x in winners], score

In [8]:
game = Game(424, 71482).play()
game.winner

('Player 160', 408679)

In [9]:
%timeit Game(424, 7148).play()

10 loops, best of 3: 45.9 ms per loop


In [10]:
%timeit Game(424, 71482).play()

1 loop, best of 3: 457 ms per loop


In [11]:
%timeit Game(424, 714820).play()

1 loop, best of 3: 4.81 s per loop


In [12]:
game = Game(424, 7148200).play()
game.winner

('Player 306', 3443939356)